In [ ]:
#Import dependencies
from pathlib import Path
import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import sqlite3



In [ ]:
# Read in csv file
crime_data_df = pd.read_csv('crime_incident_data.csv')
crime_data_df.head()

In [ ]:
# Identify any missing data and drop if needed
crime_data_df.count()

In [ ]:
# Convert Date column to datetime
crime_data_df['DATE'] = pd.to_datetime(crime_data_df['DATE'])
crime_data_df.head()

In [ ]:
# Add Year column and pull year from Date column
crime_data_df['YEAR'] = crime_data_df['DATE'].dt.year
crime_data_df.head()

In [ ]:
# Add Month column and pull month from Date column
crime_data_df['MONTH'] = crime_data_df['DATE'].dt.month_name()
crime_data_df.head()

In [ ]:
# Split Neighborhood name Column
crime_data_df[['NEW_NEIGHBORHOOD_NUMBER', 'NEIGHBORHOOD']] = crime_data_df.NEIGHBORHOOD_NAME.str.split("-", n=1, expand= True)
crime_data_df.head()

In [ ]:
# Retreive unique Incidents from Incident column
unique_incidents = crime_data_df['INCIDENT'].unique()
unique_incidents

In [ ]:
# Clean up Incident designations. 
# Replace 'Agg. Assault Dom.' and 'Agg. Assault Dom' with 'Aggravated Assault Domestic', 
# Replace 'Theft' and 'THEFT' with 'Theft'
# Replace 'Simple Assault Dom.' and 'Simple Assault Dom' with 'Simple Assault Domestic'
# Replace 'Community Engagement Event' and 'Community Event' with 'Community Event'
# Replace 'Proactive Police Visit' and 'Proactive Foot Patrol' with 'Proactive Patrol'

crime_data_df['INCIDENT'] = crime_data_df['INCIDENT'].replace({'Agg. Assault Dom.': 'Aggravated Assault Domestic', 'Agg. Assault Dom': 'Aggravated Assault Domestic',
                                                               'THEFT': 'Theft', 'Simple Asasult Dom.': 'Simple Assault Domestic', 'Simple Assault Dom': 'Simple Assault Domestic',
                                                               'Community Engagement Event': 'Community Event', 'Proactive Police Visit': 'Proactive Patrol',
                                                               'Proactive Foot Patrol': 'Proactive Patrol', 'Agg. Assault': 'Aggravated Assault',
                                                               '0': 'Other'})
crime_data_df.head()

In [ ]:
# Check that Incident types were replaced
new_unique = crime_data_df['INCIDENT'].unique()
new_unique

In [ ]:
# Drop Unnecessary columns
crime_data_df = crime_data_df[['DATE', 'YEAR', 'MONTH', 'INCIDENT', 'NEIGHBORHOOD']]
crime_data_df.head()


In [ ]:
#establish connection to and create SQLite database
conn = sqlite3.connect('stp_crime.db')

In [ ]:
#write the DataFrame to the database and create table
crime_table = 'stp_crime'
crime_data_df.to_sql(crime_table, conn, if_exists='replace', index=False)


In [ ]:
#Commit and close the connection
conn.commit()
conn.close()